In [9]:
import sys
from pathlib import Path

script_dir = Path().resolve()
root_dir = (script_dir.parent)
sys.path.append(str(root_dir))

import pandas as pd
import numpy as np

from endata.datasets.timeseries_dataset import TimeSeriesDataset
from endata.trainer import Trainer

## Training a model from scratch ##

To train your own model from scratch, the ` Trainer ` class provides a simple implementation. Simply define your a custom dataset and a ` Trainer ` object, and call the ` Trainer ` 's ` fit() ` method.

## Writing a custom dataset ##

When creating a custom time series dataset class for use with EnData, the class must inherit from the provided `TimeSeriesDataset` base class. The `TimeSeriesDataset` class provides a robust and modular framework for handling wide-format time series data. Custom implementations only need to assign a name to `self.name` and implement the `_preprocess_data` method, which is an abstract method in the base class. This method should ensure that the data is available in a clean wide-format data frame, that has the structure outlined below.

### Responsibilities of `_preprocess_data`

- Preprocess raw input data into a DataFrame that satisfies the expected structure.
- Ensure time series columns contain arrays of the correct sequence length (`seq_len`).
- Add any additional columns, such as entity identifiers or context variables.

### Benefits of the Base Class

- **Normalization and Scaling:** Automatically handles standardization and min-max scaling.
- **context Variables:** Provides support for encoding and managing context variables.
- **Time Series Merging and Splitting:** Facilitates operations to merge multiple time series columns into a single multidimensional array and split them back when needed.
- **Data Transformation:** Includes functions for inverse transformations to revert normalized data to its original scale.

---

### Expected Input DataFrame Structure

The input to the `TimeSeriesDataset` class must adhere to the following structure:

| **Column Name**       | **Description**                                                                                     |
|------------------------|-----------------------------------------------------------------------------------------------------|
| `timeseries_col1`      | A column containing arrays of length `seq_len` (after preprocessing) representing the first dimension of the time series. |
| `timeseries_col2`      | A column containing arrays of length `seq_len` (after preprocessing) representing the second dimension of the time series.|
| `entity_column`        | A column containing unique identifiers for each entity (e.g., user, household, or device ID).       |
| `context_var1`    | An (optional) static or numeric context variable (e.g., categorical or continuous feature).                |
| `context_var2`    | Further (optional) static or numeric context variables.                                                    |

- The `time_series_column_names` parameter specifies which columns are part of the time series.
- The `entity_column_name` parameter identifies the column containing unique entity IDs.
- The `context_var_column_names` parameter defines additional context variables.

---

In [10]:
class CustomTimeSeriesDataset(TimeSeriesDataset):
    """
    A custom TimeSeriesDataset implementation for handling toy data.

    Input data structure:
    - time_series_col1, time_series_col2: Time series data with arrays of length seq_len.
    - entity_id: Unique identifier for each entity.
    - static_context: Categorical or numeric context variable.
    """
    def __init__(
        self,
        data: pd.DataFrame,
        seq_len: int = 16,
        normalize: bool = True,
        scale: bool = True,
    ):
        time_series_column_names = ["time_series_col1", "time_series_col2"]
        context_var_column_names = ["context_var"]

        super().__init__(
            data=data,
            time_series_column_names=time_series_column_names,
            context_var_column_names=context_var_column_names,
            seq_len=seq_len,
            normalize=normalize,
            scale=scale,
        )

    def _preprocess_data(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Preprocesses the raw input data to ensure it conforms to the expected format.

        - Ensures time series columns contain arrays of length seq_len.
        - Ensures all required columns are present.

        Args:
            data (pd.DataFrame): The raw input data.

        Returns:
            pd.DataFrame: The preprocessed data.
        """
        required_columns = ["time_series_col1", "time_series_col2", "context_var"]
        for col in required_columns:
            if col not in data.columns:
                raise ValueError(f"Missing required column: {col}")

        for col in ["time_series_col1", "time_series_col2"]:
            data[col] = data[col].apply(
                lambda x: np.array(x).reshape(-1, 1) if isinstance(x, list) else x
            )
            data[col] = data[col].apply(
                lambda x: np.array(x) if isinstance(x, np.ndarray) else ValueError(f"Invalid data in {col}")
            )
        for col in ["time_series_col1", "time_series_col2"]:
            data[col] = data[col].apply(
                lambda x: x[:self.seq_len] if len(x) >= self.seq_len else ValueError(f"Sequence too short in {col}")
            )
        return data

Now that we have defined our dataset class, let's create some artificial timeseries columns and context variables which will comprise our dataset:

In [11]:
data = pd.DataFrame({
        "time_series_col1": [np.random.rand(16) for _ in range(100)],
        "time_series_col2": [np.random.rand(16) for _ in range(100)],
        "context_var": np.random.choice(["a", "b", "c"], size=100).tolist(),
    })

custom_dataset = CustomTimeSeriesDataset(data)
custom_dataset.data

[EnData] Loaded normalizer from /home/fuest/.cache/endata/checkpoints/custom/normalizer/custom_normalizer_dim2.ckpt


,index,context_var,timeseries,is_frequency_rare,cluster,is_pattern_rare,is_rare
0,0,2,"[[0.77760565, 0.87103003], [0.11325499, 0.6802...",False,7,True,False
1,1,1,"[[0.51916474, 0.49785388], [0.7568429, 0.90242...",False,5,False,False
2,2,2,"[[0.6528517, 0.8934492], [0.07006582, 0.694379...",False,5,False,False
3,3,0,"[[0.96020216, 0.7788653], [0.68686104, 0.99778...",True,1,True,True
4,4,0,"[[0.51167774, 0.633585], [0.82306135, 0.176096...",True,8,True,True
...,...,...,...,...,...,...,...
95,95,1,"[[0.32992682, 0.9944074], [0.359938, 0.5865123...",False,5,False,False
96,96,1,"[[0.31565368, 0.11548374], [0.76430124, 0.4452...",False,4,True,False
97,97,1,"[[0.035136953, 0.09197914], [0.639684, 0.47911...",False,0,True,False
98,98,2,"[[0.12941942, 0.5271707], [0.7092574, 0.009986...",False,5,False,False


We will now create a `Trainer` object by passing the name of the desired model and the dataset object. To start training, simply call `Trainer.fit()`.

In [12]:
trainer = Trainer(model_name="acgan", dataset=custom_dataset, overrides=["trainer.max_epochs=5", "trainer.strategy=auto"])
trainer.fit()

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]



  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | context_module | ContextModule     | 4.3 K  | train
1 | generator      | Generator         | 309 K  | train
2 | discriminator  | Discriminator     | 167 K  | train
3 | adv_loss       | BCEWithLogitsLoss | 0      | train
4 | aux_loss       | CrossEntropyLoss  | 0      | train
-------------------------------------------------------------
477 K     Trainable params
0         Non-trainable params
477 K     Total params
1.909     Total estimated model params size (MB)
37        Modules in train mode
0         Modules in eval mode


Epoch 4: 100%|██████████| 1/1 [00:00<00:00,  5.12it/s, loss_G=2.000, loss_D=3.540]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:00<00:00,  3.88it/s, loss_G=2.000, loss_D=3.540]


Once training is complete, we can create a data generator object that has access to the trained model and dataset information. To generate data, there is no need to load in a trained model. Simply define the context variables, and call the `DataGenerator` 's `generate()` method.

In [13]:
data_generator = trainer.get_data_generator()

In [14]:
data_generator.set_context(context_var=2)
generated_df = data_generator.generate(n=100)
generated_df

,context_var,time_series_col1,time_series_col2
0,2,"[0.5199467, 0.44614255, 0.5605779, 0.61748564,...","[0.3560219, 0.4248352, 0.39010537, 0.54013175,..."
1,2,"[0.52375543, 0.49376857, 0.571702, 0.59647495,...","[0.37484938, 0.3898486, 0.3503686, 0.4478007, ..."
2,2,"[0.5537223, 0.44612423, 0.5534694, 0.44235688,...","[0.37444115, 0.42353788, 0.4217857, 0.5409928,..."
3,2,"[0.54556465, 0.5067076, 0.53900754, 0.6433268,...","[0.3973128, 0.3578704, 0.403728, 0.62896526, 0..."
4,2,"[0.5280727, 0.51212806, 0.55282754, 0.48678616...","[0.40532577, 0.3838707, 0.3944916, 0.5150177, ..."
...,...,...,...
95,2,"[0.5011872, 0.45941105, 0.60996777, 0.5618797,...","[0.37103486, 0.4609786, 0.40054914, 0.55449414..."
96,2,"[0.5197611, 0.5159052, 0.57987845, 0.5982913, ...","[0.42568812, 0.41413686, 0.41159162, 0.5551385..."
97,2,"[0.5599183, 0.46095458, 0.4788934, 0.5123849, ...","[0.36776042, 0.38429236, 0.36285537, 0.5079919..."
98,2,"[0.5076361, 0.5011982, 0.52952516, 0.51724523,...","[0.37531203, 0.3710489, 0.39397565, 0.55836743..."


# Model Evaluation

The `Evaluator` class provides functionality to assess the quality of generated data compared to the original training data. It computes various metrics including:

- Distribution similarity between real and generated data
- Utility metrics
- Context-FID

To evaluate the trained model, we can use the `evaluate()` method of the `Trainer`. This method accepts:

- A dataset to evaluate against (typically the training dataset)
- Optional evaluation configuration parameters

The evaluation results provide insights into how well the model captures the underlying data patterns and maintains the relationship with context variables.

Let's run an evaluation on our trained model:

In [15]:
results = trainer.evaluate()
metadata = results["metadata"]
metrics = results["metrics"]

[EnData] Training Predictive Score Model: 100%|██████████| 5000/5000 [00:48<00:00, 103.58it/s]


In [8]:
metrics

{'DTW': {'mean': np.float64(1.4436083177538586),
  'std': np.float64(0.1508878089963357)},
 'MMD': {'mean': np.float64(0.016698863591929142),
  'std': np.float64(0.01052275694130357)},
 'Context_FID': np.float64(4.310820975640927),
 'Disc_Score': np.float64(0.44999999999999996),
 'Pred_Score': 0.25146197870373727}